<a href="https://colab.research.google.com/github/shivaniberi/StockAnalysis/blob/main/StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
vantage_api_key = userdata.get('API_Vantage')


In [ ]:
user_id = userdata.get('snowflake_userid')
password = userdata.get('snowflake_password')
account = userdata.get('snowflake_account')

In [ ]:
symbol = symbol = "MSFT"

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={vantage_api_key }'


In [ ]:
import requests  # Import the requests library
r = requests.get(url)

In [ ]:
data = r.json() # Convert the data into JSON format

In [ ]:
import requests
from datetime import datetime, timedelta

def return_last_90d_price(symbol):
    """
    - return the last 90 days of the stock prices of symbol as a list of json strings
    """
    vantage_api_key = userdata.get('API_Vantage')  # Assuming userdata is already defined
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={ vantage_api_key}'

    r = requests.get(url)
    data = r.json()

    results = []  # List to hold the last 90 days of stock info
    ninety_days_ago = datetime.today() - timedelta(days=90)  # Get the date 90 days ago

    # Iterate through the daily data and filter for the last 90 days along with date info
    for d in data["Time Series (Daily)"]:
        date_obj = datetime.strptime(d, "%Y-%m-%d")
        if date_obj >= ninety_days_ago:
          # append the data in clear format using table or another result dictionary
          price_data={

              "date":d,
              "open":data["Time Series (Daily)"][d]["1. open"],
              "high":data["Time Series (Daily)"][d]["2. high"],
              "low":data["Time Series (Daily)"][d]["3. low"],
              "close":data["Time Series (Daily)"][d]["4. close"],
              "volume":data["Time Series (Daily)"][d]["5. volume"],
              "symbol":symbol
          }

          results.append(price_data)

    return results


In [ ]:
price_list=return_last_90d_price("MSFT")

In [ ]:
len(price_list)

62

In 2024, U.S. stock markets are open Monday through Friday, excluding weekends and market holidays. From June to September 2024, the major holidays to consider are:


Independence Day: July 4, 2024 (market closed)
Labor Day: September 2, 2024 (market closed)
Excluding these holidays and weekends, you can calculate the number of trading days over the 90-day period, which would likely fall between 60 and 65 trading days.

Defining ETL functions

In [ ]:
import requests

def extract(url):
    f = requests.get(url)
    return (f.text)
    print(f.text)

In [ ]:
def transform(text):
    lines = text.strip().split("\n")
    records = []
    for l in lines:
      (date,open,high,low,volume) = l.split(",")
      records.append([date,open,high,low,volume])
    # return the records except the first entry
    return records


In [ ]:
pip install snowflake-connector-python


After installing establist connection to Snowflake

In [ ]:
import snowflake.connector

def return_snowflake_conn():
    # Establish a connection to Snowflake
    conn = snowflake.connector.connect(
        user=user_id,          # Replace with your actual user
        password=password,  # Replace with your actual password
        account=account,    # e.g., 'xyz12345.eu-central-2'
        warehouse='compute_wh',
        database='demoAPI',
        schema='raw_data',
        role='accountadmin',
        ocsp_fail_open=False
    )
 #create a cursor object so that sql knows the starting point of executiom of commands
    return conn.cursor()

In [ ]:
# This function returns a list of price data for the last 90 days
price_list = return_last_90d_price("MSFT")  # This should give you the structured data

def load(con, records):
    # Check if records is empty
    if not records:
        print("No records to load.")
        return  # Exit if there are no records

    # Define the target table for price data
    target_table = "demoAPI.raw_data.stock_price"

    # Create the table if it does not exist
    conn.execute(f"""
    CREATE or Replace TABLE {target_table} (
      date DATE Primary Key,
      symbol VARCHAR,
      open NUMBER,
      high NUMBER,
      low NUMBER,
      close NUMBER,
      volume NUMBER
    )""")

    # Load records into the table
    for r in records:
        date = r['date']
        symbol = r['symbol']
        open_price = r['open']
        high_price = r['high']
        low_price = r['low']
        close_price = r['close']
        volume = r['volume']


        print(f"Inserting data for {date}: Open={open_price}, Symbol='{symbol}', High={high_price}, Low={low_price}, Close={close_price}, Volume={volume}")

        # Use parameterized INSERT INTO to avoid SQL injection and special character issues
        sql = f"""
        INSERT INTO {target_table} (date, symbol, open, high, low, close, volume)
        VALUES (TO_DATE('{date}', 'YYYY-MM-DD'),'{symbol}', {open_price}, {high_price}, {low_price}, {close_price}, {volume})
        """

        conn.execute(sql)

# Get the Snowflake connection
conn = return_snowflake_conn()

# Load the data into the Snowflake table
load(conn, price_list)


Inserting data for 2024-09-26: Open=435.0000, Symbol='MSFT', High=435.2400, Low=429.1300, Close=431.3100, Volume=14446507
Inserting data for 2024-09-25: Open=429.8300, Symbol='MSFT', High=433.1190, Low=428.5700, Close=432.1100, Volume=13396364
Inserting data for 2024-09-24: Open=433.0000, Symbol='MSFT', High=433.3500, Low=426.1000, Close=429.1700, Volume=17015754
Inserting data for 2024-09-23: Open=434.2800, Symbol='MSFT', High=436.4600, Low=430.3889, Close=433.5100, Volume=15128891
Inserting data for 2024-09-20: Open=437.2200, Symbol='MSFT', High=439.2400, Low=434.2200, Close=435.2700, Volume=55167106
Inserting data for 2024-09-19: Open=441.2250, Symbol='MSFT', High=441.5000, Low=436.9000, Close=438.6900, Volume=21706559
Inserting data for 2024-09-18: Open=435.0000, Symbol='MSFT', High=436.0300, Low=430.4100, Close=430.8100, Volume=18898042
Inserting data for 2024-09-17: Open=440.2300, Symbol='MSFT', High=441.8500, Low=432.2700, Close=435.1500, Volume=18874231
Inserting data for 2024-

Using try/except in below block

In [ ]:
# This function returns a list of price data for the last 90 days
price_list = return_last_90d_price("MSFT")  # This should give you the structured data

def load(con, records):
    # Check if records is empty
    if not records:
        print("No records to load.")
        return  # Exit if there are no records

    # Define the target table for price data
    target_table = "demoAPI.raw_data.stock_price"

    try:
        # Start the transaction
        con.execute("BEGIN;")

        # Create or replace the table
        con.execute(f"""
        CREATE OR REPLACE TABLE {target_table} (
          date DATE Primary Key,
          symbol VARCHAR,
          open NUMBER,
          high NUMBER,
          low NUMBER,
          close NUMBER,
          volume NUMBER
        )
        """)

        # Load records into the table
        for r in records:
            date = r['date']
            open_price = r['open']
            high_price = r['high']
            low_price = r['low']
            close_price = r['close']
            volume = r['volume']

            print(f"Inserting data for {date}: Open={open_price}, High={high_price}, Low={low_price}, Close={close_price}, Volume={volume}")

            # Use parameterized INSERT INTO to avoid SQL injection and special character issues
            sql = f"""
            INSERT INTO {target_table} (date, symbol, open, high, low, close, volume)
            VALUES (TO_DATE('{date}', 'YYYY-MM-DD'),'{symbol}', {open_price}, {high_price}, {low_price}, {close_price}, {volume})
            """

            con.execute(sql)

        # Commit the transaction
        con.execute("COMMIT;")

    except Exception as e:
        # Rollback the transaction in case of an error
        con.execute("ROLLBACK;")
        print("An error occurred while loading data:", e)
        raise e  # Re-raise the exception for further handling if needed

# Get the Snowflake connection
conn = return_snowflake_conn()

# Load the data into the Snowflake table
load(conn, price_list)




Inserting data for 2024-09-25: Open=429.8300, High=433.1190, Low=428.5700, Close=432.1100, Volume=13396364
Inserting data for 2024-09-24: Open=433.0000, High=433.3500, Low=426.1000, Close=429.1700, Volume=17015754
Inserting data for 2024-09-23: Open=434.2800, High=436.4600, Low=430.3889, Close=433.5100, Volume=15128891
Inserting data for 2024-09-20: Open=437.2200, High=439.2400, Low=434.2200, Close=435.2700, Volume=55167106
Inserting data for 2024-09-19: Open=441.2250, High=441.5000, Low=436.9000, Close=438.6900, Volume=21706559
Inserting data for 2024-09-18: Open=435.0000, High=436.0300, Low=430.4100, Close=430.8100, Volume=18898042
Inserting data for 2024-09-17: Open=440.2300, High=441.8500, Low=432.2700, Close=435.1500, Volume=18874231
Inserting data for 2024-09-16: Open=430.6000, High=433.5300, Low=428.2200, Close=431.3400, Volume=13834697
Inserting data for 2024-09-13: Open=425.8250, High=431.8300, Low=425.4600, Close=430.5900, Volume=15874555
Inserting data for 2024-09-12: Open=4

Ensuring Idempotency by running the load function twice:


In [ ]:
def check_record_count(con):
    target_table = "demoAPI.raw_data.stock_price"
    result = con.execute(f"SELECT COUNT(*) FROM {target_table}").fetchone()
    return result[0]

In [ ]:
# Get Snowflake connection
conn = return_snowflake_conn()

# Check initial record count
initial_count = check_record_count(conn)
print(f"Initial record count: {initial_count}")

# Load data twice
load(conn, price_list)
first_load_count = check_record_count(conn)
print(f"Record count after first load: {first_load_count}")

load(conn, price_list)
second_load_count = check_record_count(conn)
print(f"Record count after second load: {second_load_count}")

# Validate idempotency
if first_load_count == second_load_count:
    print("Idempotent: record counts are consistent.")
else:
    print("Not idempotent: record counts differ.")

Initial record count: 61
Inserting data for 2024-09-25: Open=429.8300, Symbol='MSFT', High=433.1190, Low=428.5700, Close=432.1100, Volume=13396364
Inserting data for 2024-09-24: Open=433.0000, Symbol='MSFT', High=433.3500, Low=426.1000, Close=429.1700, Volume=17015754
Inserting data for 2024-09-23: Open=434.2800, Symbol='MSFT', High=436.4600, Low=430.3889, Close=433.5100, Volume=15128891
Inserting data for 2024-09-20: Open=437.2200, Symbol='MSFT', High=439.2400, Low=434.2200, Close=435.2700, Volume=55167106
Inserting data for 2024-09-19: Open=441.2250, Symbol='MSFT', High=441.5000, Low=436.9000, Close=438.6900, Volume=21706559
Inserting data for 2024-09-18: Open=435.0000, Symbol='MSFT', High=436.0300, Low=430.4100, Close=430.8100, Volume=18898042
Inserting data for 2024-09-17: Open=440.2300, Symbol='MSFT', High=441.8500, Low=432.2700, Close=435.1500, Volume=18874231
Inserting data for 2024-09-16: Open=430.6000, Symbol='MSFT', High=433.5300, Low=428.2200, Close=431.3400, Volume=13834697